# 模型的推理

推理在深度学习中表示模型的预测过程。一般来说，推理会使用pipeline（串行管道过程）来执行所需要的操作。一个完整的pipeline一般包括了数据的前处理、
模型的前向推理、数据的后处理三个过程。

# Pipline介绍

pipeline()方法是ModelScope框架上最基础的用户方法之一，可对多种领域的多种模型进行快速推理。通过pipeline()方法，用户可以只需要一行代码即可完成对特定任务的模型推理。
ModelScope中pipeline的预处理过程复用了[预处理器](./数据的预处理.ipynb)。由于预处理器支持传入mode参数及configuration中的train/val两种独立的预处理过程，因此您可以在预处理过程中
方便地区分处理训练、评估、推理三种情况(评估和推理会复用val过程)。pipeline的模型前向推理过程调用了传入模型的forward方法和模型的postprocess方法，以便返回值不标准的模型可以通过定制化的postprocess
方法回传pipeline适配的返回值。pipeline的postprocess会在模型的postprocess方法之后被调用，它返回了最终展现给用户的值。

# Pipeline的使用
本文简单介绍如何使用`pipeline`方法加载模型进行推理。`pipeline`方法支持按照任务类型、模型名称从模型仓库拉取模型进行进行推理，包含以下几个方面：

- 使用pipeline()函数进行推理
- 指定特定预处理、特定模型进行推理
- 不同场景推理任务示例
## 环境准备
详细步骤可以参考 [环境安装指南](../快速入门/环境安装.ipynb)。
## Pipeline基本用法
下面以中文分词任务为例，说明pipeline函数的基本用法。

1.  pipeline函数支持指定特定任务名称，加载任务默认模型，创建对应pipeline对象。
执行如下python代码 ：


In [1]:
from modelscope.pipelines import pipeline
word_segmentation = pipeline('word-segmentation')


 

2.  输入文本 


In [1]:
input_str = '今天天气不错，适合出去游玩'
print(word_segmentation(input_str))
{'output': '今天 天气 不错 ， 适合 出去 游玩'}


 

3.  输入多条样本 

pipeline对象也支持传入多个样本列表输入，返回对应输出列表，每个元素对应输入样本的返回结果。



In [1]:
inputs =  ['今天天气不错，适合出去游玩','这本书很好，建议你看看']
print(word_segmentation(inputs))
[{'output': '今天 天气 不错 ， 适合 出去 游玩'}, {'output': '这 本 书 很 好 ， 建议 你 看看'}]



4. 输入一个数据集



In [1]:
from modelscope.msdatasets import MsDataset
from modelscope.pipelines import pipeline

inputs = ['今天天气不错，适合出去游玩', '这本书很好，建议你看看']
dataset = MsDataset.load(inputs, target='sentence')
word_segmentation = pipeline('word-segmentation')
outputs = word_segmentation(dataset)
for o in outputs:
    print(o)

# 输出
{'output': '今天 天气 不错 ， 适合 出去 游玩'}
{'output': '这 本 书 很 好 ， 建议 你 看看'}



## 指定预处理、模型进行推理
pipeline函数支持传入实例化的预处理对象、模型对象，从而支持用户在推理过程中定制化预处理、模型。

1. 创建模型对象并使用指定模型进行推理


In [1]:
from modelscope.models import Model
from modelscope.pipelines import pipeline

model = Model.from_pretrained('damo/nlp_structbert_word-segmentation_chinese-base')
word_segmentation = pipeline('word-segmentation', model=model)
input = '今天天气不错，适合出去游玩'
print(word_segmentation(input))
{'output': '今天 天气 不错 ， 适合 出去 游玩'}



2. 指定tokenizer预处理，并用已指定的模型和预处理方法进行推理



In [1]:
from modelscope.models import Model
from modelscope.pipelines import pipeline
from modelscope.preprocessors import TokenClassificationPreprocessor

model = Model.from_pretrained('damo/nlp_structbert_word-segmentation_chinese-base')
tokenizer = TokenClassificationPreprocessor(model.model_dir)
word_segmentation = pipeline('word-segmentation', model=model, preprocessor=tokenizer)
input = '今天天气不错，适合出去游玩'
print(word_segmentation(input))
{'output': '今天 天气 不错 ， 适合 出去 游玩'}



## 不同场景任务推理pipeline使用示例
### 图像
以人像抠图（'portrait-matting'）为例
输入图片：

![image_matting.png](./resources/1656989748829-9ab3aa9b-461d-44f8-98fb-c85bc6f670f9.png)



In [1]:
import cv2
from modelscope.pipelines import pipeline

portrait_matting = pipeline('portrait-matting')
result = portrait_matting('https://modelscope.oss-cn-beijing.aliyuncs.com/test/images/image_matting.png')
cv2.imwrite('result.png', result['output_img'])


输出：
![result.png](./resources/1656989768092-5470f8ac-cda8-4703-ac98-dbb6fd675b34.png)
### 
### 语音
以（'text-to-speech'）为例（注意，当前ModelScope版本tts能力体验依赖python3.7，linux环境，后续会进行扩展）


In [1]:
from scipy.io.wavfile import write
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from modelscope.outputs import OutputKeys


text = '今天北京天气怎么样？'
voice = 'zhitian_emo'

sambert_hifigan_tts =pipeline(
    task=Tasks.text_to_speech, model='damo/speech_sambert-hifigan_tts_zhizhe_emo_zh-cn_16k')
output = sambert_hifigan_tts(input=text, voice=voice)
pcm = output[OutputKeys.OUTPUT_PCM]
write('output.wav', 16000, pcm)


### 多模态
以多模态表征模型为例：



In [1]:
from modelscope.models import Model
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
from PIL import Image
import requests

model = Model.from_pretrained('damo/multi-modal_gemm-vit-large-patch14_generative-multi-modal-embedding')
p = pipeline(task=Tasks.generative_multi_modal_embedding, model=model)

url = 'http://clip-multimodal.oss-cn-beijing.aliyuncs.com/lingchen/demo/dogs.jpg'
image = Image.open(requests.get(url, stream=True).raw)
text = 'dogs playing in the grass'

img_embedding = p.forward({'image': image})['img_embedding']
print('image embedding: {}'.format(img_embedding))

text_embedding = p.forward({'text': text})['text_embedding']
print('text embedding: {}'.format(text_embedding))

image_caption = p.forward({'image': image, 'captioning': True})['caption']
print('image caption: {}'.format(image_caption))



# 当前支持的Task列表
以下给出当前支持的 task 类型列表（字符串），以及相应的别名（便于管理）。在ModelScope提供的API中，可直接使用字符串作为task入参，比如


In [1]:
from modelscope.pipelines import pipeline
pipe = pipeline('portrait-matting')


也可以使用Tasks.别名，比如


In [1]:
from modelscope.utils.constant import Tasks
pipe = pipeline(Tasks.portrait_matting)


更多的任务支持也在**不断扩展**中。大家可以通过ModelScope页面上的任务分类来选择心仪的模型，也可以从具体模型的页面上，来获取每个模型使用时对应的范例代码。
### 视觉
```jsx
    # ocr
    ocr_detection = 'ocr-detection'
    ocr_recognition = 'ocr-recognition'

    # human face body related
    animal_recognition = 'animal-recognition'
    face_detection = 'face-detection'
    face_recognition = 'face-recognition'
    human_detection = 'human-detection'
    human_object_interaction = 'human-object-interaction'
    face_image_generation = 'face-image-generation'
    body_2d_keypoints = 'body-2d-keypoints'
    general_recognition = 'general-recognition'

    image_classification = 'image-classification'
    image_multilabel_classification = 'image-multilabel-classification'
    image_classification_imagenet = 'image-classification-imagenet'
    image_classification_dailylife = 'image-classification-dailylife'

    image_object_detection = 'image-object-detection'

    image_segmentation = 'image-segmentation'
    portrait_matting = 'portrait-matting'
    text_driven_segmentation = 'text-driven-segmentation'
    shop_segmentation = 'shop-segmentation'

    # image editing
    skin_retouching = 'skin-retouching'
    image_super_resolution = 'image-super-resolution'
    image_colorization = 'image-colorization'
    image_color_enhancement = 'image-color-enhancement'
    image_denoising = 'image-denoising'
    image_portrait_enhancement = 'image-portrait-enhancement'

    # image generation
    image_to_image_translation = 'image-to-image-translation'
    image_to_image_generation = 'image-to-image-generation'
    image_style_transfer = 'image-style-transfer'
    image_portrait_stylization = 'image-portrait-stylization'

    image_embedding = 'image-embedding'

    product_retrieval_embedding = 'product-retrieval-embedding'

    # video recognition
    live_category = 'live-category'
    action_recognition = 'action-recognition'
    action_detection = 'action-detection'
    video_category = 'video-category'
    video_embedding = 'video-embedding'
    virtual_try_on = 'virtual-try-on'
    crowd_counting = 'crowd-counting'
    movie_scene_segmentation = 'movie-scene-segmentation'

    # video editing
    video_inpainting = 'video-inpainting'

    # reid and tracking
    video_single_object_tracking = 'video-single-object-tracking'
    video_summarization = 'video-summarization'
    image_reid_person = 'image-reid-person'
```
### 自然语言处理
```jsx
    # nlp tasks
    word_segmentation = 'word-segmentation'
    part_of_speech = 'part-of-speech'
    named_entity_recognition = 'named-entity-recognition'
    nli = 'nli'
    sentiment_classification = 'sentiment-classification'
    sentiment_analysis = 'sentiment-analysis'
    sentence_similarity = 'sentence-similarity'
    text_classification = 'text-classification'
    sentence_embedding = 'sentence-embedding'
    passage_ranking = 'passage-ranking'
    relation_extraction = 'relation-extraction'
    zero_shot = 'zero-shot'
    translation = 'translation'
    token_classification = 'token-classification'
    conversational = 'conversational'
    text_generation = 'text-generation'
    task_oriented_conversation = 'task-oriented-conversation'
    dialog_intent_prediction = 'dialog-intent-prediction'
    dialog_state_tracking = 'dialog-state-tracking'
    table_question_answering = 'table-question-answering'
    sentence_embedding = 'sentence-embedding'
    fill_mask = 'fill-mask'
    summarization = 'summarization'
    question_answering = 'question-answering'
    zero_shot_classification = 'zero-shot-classification'
    backbone = 'backbone'
    text_error_correction = 'text-error-correction'
    faq_question_answering = 'faq-question-answering'
    conversational_text_to_sql = 'conversational-text-to-sql'
    information_extraction = 'information-extraction'
    document_segmentation = 'document-segmentation'
```

下面我们详细介绍几个常用NLP任务的推理过程。

#### 文本分类任务的推理

ModelScope的文本分类任务有：

| 任务编码                     |          任务名称 |
|--------------------------|--------------:|
| nli                      |        自然语言推理 |
| sentiment-classification |          情感分析 |
| sentence-similarity      |         句子相似度 |
| zero-shot-classification | zero-shot分类任务 |
| text-classification      |        一般分类任务 |

一般来说表中其他任务可以被视为text-classification的子级任务。由于模型可能存在特异性，因此ModelScope目前让这些任务并列存在，以达到模型和推理的灵活性。
在用户使用的时候可以按照子级别任务使用，如果没有明确的子级别任务，可以直接使用text-classification任务。

代码样例：


In [1]:
from modelscope.models import Model
from modelscope.preprocessors import Preprocessor
model = Model.from_pretrained('damo/nlp_structbert_sentence-similarity_chinese-base')
preprocessor = Preprocessor.from_pretrained('damo/nlp_structbert_sentence-similarity_chinese-base')
from modelscope.pipelines import pipeline
pipeline_ins = pipeline('text-classification', model=model, preprocessor=preprocessor)
print(pipeline_ins(('这是个测试', '这也是个测试')))



在pipeline中，上述的所有任务都使用同一个pipeline：`TextClassificationPipeline`。这个pipeline的构造参数包括：
- model 一个模型id或包含模型的本地路径或一个模型实例。模型实例可以是Model的子类，或一个torch.nn.Module的实例。
- preprocessor 传入模型对应的预处理器。如果未传入，pipeline会尝试使用model.model_dir来创建一个预处理器
- first_sequence 第一段语言的key
- second_sequence 第二段语言的key
- sequence_length 支持的最大输入长度
- id2label id和label的mapping关系，如果不传入，pipeline会尝试从preprocessor获取id2label。preprocessor的id2label会从
model.model_dir的label_mapping.json或configuration.json或config.json自动获取

如果__call__方法传入了str（单句）或tuple（双句），first_sequence和second_sequence两个字段不起作用。

传入的模型可以是外部PyTorch模型，只要该模型满足：
- 模型返回值是dict，或实现了__getitem__方法，并且可以获取`logits`字段，类型是torch.Tensor
- 该模型有配套的预处理器传入，或该外部model实例有model_dir字段，其路径中存在configuration.json用以初始化一个预处理器

`TextClassificationPipeline`的返回值是：

- scores 各标签的概率值序列
- labels 各标签值的序列

上述两个序列以升序排列。

`TextClassificationPipeline`也是OFA模型的推理pipeline，但目前其返回值和上述返回值不同。

#### fill-mask任务

ModelScope的fill-mask任务的Pipeline是`FillMaskPipeline`。这个pipeline的构造参数包括：
- model 一个模型id或包含模型的本地路径或一个模型实例。模型实例可以是Model的子类，或一个torch.nn.Module的实例。
- preprocessor 传入模型对应的预处理器。如果未传入，pipeline会尝试使用model.model_dir来创建一个预处理器
- first_sequence 第一段语言的key
- sequence_length 支持的最大输入长度

如果__call__方法传入了str（单句）或tuple（双句），first_sequence字段不起作用。

传入的模型可以是外部PyTorch模型，只要该模型满足：
- 模型返回值是dict，或实现了__getitem__方法，并且可以获取`logits`字段和`input_ids`字段，其中input_ids字段代表预处理之后输入模型的token序列，类型是torch.Tensor
- 该模型有配套的预处理器传入，或该外部model实例有model_dir字段，其路径中存在configuration.json用以初始化一个预处理器

`FillMaskPipeline`的返回值是：

- text 生成的句子

代码样例：


In [1]:
from modelscope.pipelines import pipeline
pipeline_ins = pipeline('fill-mask', model='damo/nlp_structbert_fill-mask_english-large')
input = 'Everything in [MASK] you call reality is really [MASK] a reflection of your [MASK].'
print(pipeline_ins(input))



如果输入是中文，由于各输入模型的tokenizer不同，部分模型最后返回的text可能存在多余的空格。

### 语音
```jsx
    # audio tasks
    auto_speech_recognition = 'auto-speech-recognition'
    text_to_speech = 'text-to-speech'
    speech_signal_process = 'speech-signal-process'
    acoustic_echo_cancellation = 'acoustic-echo-cancellation'
    acoustic_noise_suppression = 'acoustic-noise-suppression'
    keyword_spotting = 'keyword-spotting'
```
### 多模态
```jsx
    # multi-modal tasks
    image_captioning = 'image-captioning'
    visual_grounding = 'visual-grounding'
    text_to_image_synthesis = 'text-to-image-synthesis'
    multi_modal_embedding = 'multi-modal-embedding'
    generative_multi_modal_embedding = 'generative-multi-modal-embedding'
    multi_modal_similarity = 'multi-modal-similarity'
    visual_question_answering = 'visual-question-answering'
    visual_entailment = 'visual-entailment'
    video_multi_modal_embedding = 'video-multi-modal-embedding'
    image_text_retrieval = 'image-text-retrieval'
```

